In [1]:
# Notebook: Analysis Example - MCPA 911
# Author: Thomas Purk
# Date: 2025-03-27
# Reference: 

# Analysis Example

With a best estimator model in hand from a grid search optimization step, use the model to estimate future activity, and distribute new vehicle resources accross the member townships.

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/emergency-911-calls-mcpa/911-processed.csv
/kaggle/input/emergency-911-calls-mcpa/911.csv
/kaggle/input/emergency-911-calls-mcpa/bes_rfr_model.pkl
/kaggle/input/emergency-911-calls-mcpa/911-encoded.csv
/kaggle/input/emergency-911-calls-mcpa/rfr_model.pkl


In [6]:
# Notebook Step up steps

# SciKit Learn imports
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error

# For loading models
import pickle

# Prevent Pandas and other warnings from displaying above outputcells, improve readability
import warnings
warnings.filterwarnings('ignore')

# Load visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Load the data 
df_in_path = '/kaggle/input/emergency-911-calls-mcpa/911-processed.csv'
df_911 = pd.read_csv(df_in_path)
df_911.info() # validate

# Load the model - Start here unless model needs to be recreated
with open('/kaggle/input/emergency-911-calls-mcpa/bes_rfr_model.pkl', 'rb') as f:
    rfr_model = pickle.load(f)

rfr_model

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 649696 entries, 0 to 649695
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   twp           649696 non-null  object
 1   e             649696 non-null  int64 
 2   service_type  649696 non-null  object
 3   service_desc  649696 non-null  object
 4   month         649696 non-null  int64 
 5   day_of_week   649696 non-null  int64 
 6   day_night     649696 non-null  object
dtypes: int64(3), object(4)
memory usage: 34.7+ MB


RandomForestRegressor(max_features='auto', min_samples_leaf=4,
                      min_samples_split=10, random_state=42, verbose=2)

In [7]:
# Usage Example
# Assumption - The model predicts event counts per calander year
# Assumption - Each township currently has enough vehicles to cover historical call value
# Assumption - New vehicles are applied to cover a 5% increase in each type of call volume
# Assumption - One car is needed for evey 100 calls

from scipy.optimize import linprog

In [21]:
# Prepare the data

# Create a dataframe containing the current vehicle counts
# Group the data to get unique combinations of the following columns.
# NOTE: The model was trained on these categories
# This will goal is to have the model predict how many calls of each category
# will be received for the coming year

# Columns of interest 
cols = ['twp', 'month', 'day_of_week','day_night', 'service_type', 'service_desc']
df_911_sub = df_911[cols]

# Grouping - This dataframe represents the future records we need to predict call counts for
group_911 = df_911_sub.groupby(cols)
# Group then drop call_count. WE
df_911_grouped = group_911.size().reset_index(name='legacy_call_count')
df_911_grouped.drop(
    labels='legacy_call_count',
    axis='columns',
    inplace=True
)

# One-hot encode categorical features
encoder = OneHotEncoder(drop='first', sparse=False)
encoded_features = encoder.fit_transform(df_911_grouped)

# Convert encoded features to a DataFrame
df_911_encoded = pd.DataFrame(
    data=encoded_features, 
    columns=encoder.get_feature_names_out()
)


In [56]:
# Make prediction for next year's call counts
predicted_calls = rfr_model.predict(df_911_encoded)

# Assume call volume will grow by 5% accross the board
# The analysis should determine how to distribute new vehicles to cover new calls
# Append the prediction back to the data
df_911_grouped['predicted_call_growth'] = (predicted_calls * 0.10)

[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    1.9s


In [57]:
# Arrange the data for the analysis

# Group by  twp, service_type and sum predicted_call_growth
df_demand_growth = df_911_grouped[['twp','service_type','predicted_call_growth']]
# Grouping
df_demand_growth = df_demand_growth.groupby(['twp','service_type']).sum().reset_index()
df_demand_growth

,twp,service_type,predicted_call_growth
0,ABINGTON TOWNSHIP,EMS,1459.030500
1,ABINGTON TOWNSHIP,Fire,445.562216
2,ABINGTON TOWNSHIP,Traffic,936.532398
3,AMBLER BOROUGH,EMS,281.251132
4,AMBLER BOROUGH,Fire,103.290845
...,...,...,...
181,WHITPAIN TOWNSHIP,Fire,213.247122
182,WHITPAIN TOWNSHIP,Traffic,482.072155
183,WORCESTER TOWNSHIP,EMS,299.734721
184,WORCESTER TOWNSHIP,Fire,110.622800


In [71]:
# How many townships are there?
twp_count = df_911_twp_grouped['twp'].nunique()

# Call per car
calls_per_vehicle = 100

# Define supply constraints
new_vehicles_police = 50
new_vehicles_fire = 50
new_vehicles_ems = 50

# Fair discribution constraints
min_vehicles_per_twp = 0
max_vehicles_per_twp = 4

predicted_calls_ems = df_demand_growth[df_demand_growth['service_type'] == 'EMS']['predicted_call_growth']
predicted_calls_traffic = df_demand_growth[df_demand_growth['service_type'] == 'Traffic']['predicted_call_growth']
predicted_calls_fire = df_demand_growth[df_demand_growth['service_type'] == 'Fire']['predicted_call_growth']

# Calls per vehicle per year = 2016
new_vehicle_demand_ems = predicted_calls_ems / calls_per_vehicle

# Bounds for variables (x1 >= 0, x2 >= 0)
# Each township gets at least 0
x_bounds = [(min_vehicles_per_twp, max_vehicles_per_twp) for _ in range(twp_count)]

# Linear Programming to optimize allocation
res = linprog(
    c=-new_vehicle_demand_ems,  # We want to maximize allocations where demand is high
    A_ub=[ [1] * twp_count],  # Ensure all vehicles are allocated
    b_ub=[new_vehicles_ems],
    bounds=x_bounds
)

allocated_vehicles = res.x  # Optimized vehicle allocation per township

df_allocations = pd.DataFrame({ 'twp': df_911_twp_grouped['twp'].unique(), 'ems_allocation': allocated_vehicles})

print(df_allocations.to_string())

                          twp  ems_allocation
0           ABINGTON TOWNSHIP             4.0
1              AMBLER BOROUGH             0.0
2          BRIDGEPORT BOROUGH             0.0
3          BRYN ATHYN BOROUGH             0.0
4         CHELTENHAM TOWNSHIP             4.0
5        COLLEGEVILLE BOROUGH             0.0
6        CONSHOHOCKEN BOROUGH             0.0
7           DOUGLASS TOWNSHIP             0.0
8     EAST GREENVILLE BOROUGH             0.0
9      EAST NORRITON TOWNSHIP             4.0
10         FRANCONIA TOWNSHIP             0.0
11         GREEN LANE BOROUGH             0.0
12            HATBORO BOROUGH             0.0
13           HATFIELD BOROUGH             0.0
14          HATFIELD TOWNSHIP             0.0
15           HORSHAM TOWNSHIP             4.0
16         JENKINTOWN BOROUGH             0.0
17           LANSDALE BOROUGH             0.0
18          LIMERICK TOWNSHIP             0.0
19   LOWER FREDERICK TOWNSHIP             0.0
20     LOWER GWYNEDD TOWNSHIP     

In [72]:
sum(allocated_vehicles)

50.0